[View in Colaboratory](https://colab.research.google.com/github/battlerhythm/tensorflow/blob/master/5.2%20TensorBoard.ipynb)

In [0]:
!pip install -U tensorboardcolab -qq

In [2]:
from tensorboardcolab import *
import shutil

# clean out the directory
shutil.rmtree('./Graph', ignore_errors=True)
os.mkdir('./Graph')

tf.reset_default_graph()

# will start the tunneling and will print out a link:
tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://983ea7f7.ngrok.io


In [3]:
from google.colab import files
files.upload()

Saving data.csv to data (1).csv


{'data.csv': b'0, 0, 1, 0, 0\n1, 0, 0, 1, 0\n1, 1, 0, 0, 1\n0, 0, 1, 0, 0\n0, 0, 1, 0, 0\n0, 1, 0, 0, 1\n'}

In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [0]:
global_step = tf.Variable(0, trainable=False, name='gloabal_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현해줍니다.
# 변수 뒤에 name='W1'처럼 이름을 붙이면 텐서보드에서 해당 이름의 변수가 어디에서 사용되는지 쉽게 확인할 수 있습니다.
# 이름은 변수뿐만 아니라 플레이스 홀더, 각각의 연산, 활성화 함수 등 모든 텐서에 붙일 수 있습니다.
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3) #출력단에서는 보통 Activation Function을 사용하지 않습니다.
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    # 손실값을 추적하기 위해 수집할 값을 지정하는 코드를 작성합니다.
    tf.summary.scalar('cost', cost)

In [6]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

# tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음
# tf.summary.FileWriter함수를 이용해 그래프와 텐서들의 값을 저장할 디렉터리를 설정합니다.
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step), 'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    # session.run을 이용해 앞서 merged로 모아둔 텐서의 값들을 계산하여 수집한뒤,
    # writer.add_summary함수를 이용해 해당 값들을 앞서 지정한 디렉토리에 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

sess.close()

# Consol에서
# tensorboard --logdir=./logs
# output에 출력된 portnumber를 확인하고, 웹브라우저에서 열면 TensorBoard가 보입니다.

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-100
Step: 101,  Cost: 0.550
Step: 102,  Cost: 0.550
Step: 103,  Cost: 0.550
Step: 104,  Cost: 0.550
Step: 105,  Cost: 0.550
Step: 106,  Cost: 0.550
Step: 107,  Cost: 0.550
Step: 108,  Cost: 0.550
Step: 109,  Cost: 0.550
Step: 110,  Cost: 0.550
Step: 111,  Cost: 0.550
Step: 112,  Cost: 0.550
Step: 113,  Cost: 0.550
Step: 114,  Cost: 0.550
Step: 115,  Cost: 0.550
Step: 116,  Cost: 0.550
Step: 117,  Cost: 0.550
Step: 118,  Cost: 0.550
Step: 119,  Cost: 0.550
Step: 120,  Cost: 0.550
Step: 121,  Cost: 0.550
Step: 122,  Cost: 0.550
Step: 123,  Cost: 0.550
Step: 124,  Cost: 0.550
Step: 125,  Cost: 0.550
Step: 126,  Cost: 0.550
Step: 127,  Cost: 0.550
Step: 128,  Cost: 0.550
Step: 129,  Cost: 0.550
Step: 130,  Cost: 0.550
Step: 131,  Cost: 0.550
Step: 132,  Cost: 0.550
Step: 133,  Cost: 0.550
Step: 134,  Cost: 0.550
Step: 135,  Cost: 0.550
Step: 136,  Cost: 0.550
Step: 137,  Cost: 0.550
Step: 138,  Cost: 0.550
Step: 139,  Cost: 0.550
S

In [0]:
train_writer = tbc.get_writer();
train_writer.add_graph(sess.graph)

train_writer.flush();
tbc.close()